In [ ]:
работы, которые были запланированы: 
    1)  spacy NER pipeline -> будут непонятные сущности вообще все подряд, мы сразу удалим категории GPE например
    2)  spacy NER дообучить ЯП -> посмотреть какие слова попали, мб это будут технологии? 
    3)  nltk NER 
    4)  spacy Span все имена собственные
_________________
    5) руками разметить тексты 
эти размеченные тексты должны выглядеть:
set(str, str, str ...) | set() <- text
df с такими 2мя колонками text, set_of_techs
__________________
скор = len(set_true & set_pred) / len(set_true)

In [13]:
import json

import re

from collections import Counter, defaultdict

import pandas as pd

pd.options.display.max_colwidth = 3000

pd.options.display.max_columns = 3000

from functools import  reduce

import tqdm

from html import unescape

import nltk
nltk.download('maxent_ne_chunker')
nltk.download('words')

from nltk.tokenize import word_tokenize
from nltk import ne_chunk, pos_tag

from spacy.tokens import Span

import spacy

nlp = spacy.load('en_core_web_sm')

from spacy.matcher import Matcher

In [20]:
orig_data = pd.read_csv('./data/df_hni_2018_05_orig.csv')

/home/ann/venv/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [324]:
orig_data.head(1)

,by,dead,deleted,descendants,id,kids,parent,parts,poll,retrieved_on,score,text,time,title,type,url
0,burntwater,NaN,NaN,NaN,16963601,NaN,16962078.0,NaN,NaN,1528401048,NaN,"The New York State fee by itself is $200. Unless I&#x27;m missing something, you paid significantly more than that for the publication requirement (it was around $1,000, give or take a couple hundred, last I checked).",1525132803,NaN,comment,NaN


###  Через bs4 или lxml тоже пробовала удалить тэги, но остановилась на этом варианте

In [328]:
with open('data/html_tags.txt') as f:
    html_tags = f.read().split('\n')

In [161]:
def replace_tags(text):
    
    text = unescape(text)\
        .replace('<p>', ' ')\
        .replace('<pre>', ' ')\
        .replace('</pre>', ' ')
    
    for tag in html_tags:
        start_tag = '<{}'.format(tag)
        end_tag = '</{}>'.format(tag)
        pattern = start_tag + '.*' + end_tag
        text = re.sub(pattern, 'html_tag_{}'.format(tag).upper(), text, flags=re.DOTALL)
    return text

In [165]:
%%time 

fin_df['text_without_tags'] = fin_df['text'].fillna('').apply(replace_tags)

CPU times: user 45.3 s, sys: 31.8 ms, total: 45.4 s
Wall time: 45.4 s


### spacy NER

In [168]:
all_texts = list(fin_df['text_without_tags'].values)

In [ ]:
%%time

docs = list(tqdm.tqdm(nlp.pipe(all_texts), total=len(all_texts)))

fin_df['spacy_doc'] = docs

In [ ]:
fin_df['spacy_ents'] = fin_df['spacy_doc'].apply(lambda x: set([ent.text for ent in x.ents]))

### spacy PROP

In [268]:
matcher = Matcher(nlp.vocab)

pattern = [{'POS': 'PROPN', 'OP': '+'}, {'POS': 'PROPN', 'OP': '+'}]

matcher.add('prop_names', None, pattern)

In [278]:
def pron_names_spacy(doc, matcher):
    return set([doc[start : end].text for match_id, start, end in matcher(doc)])

In [280]:
fin_df['prop_names_spacy'] = fin_df['spacy_doc'].apply(lambda x: pron_names_spacy(x, matcher))

In [ ]:
# хотелось бы как-то искать версии технологий, например python 2 и python 3 различать
# но есть же еще версионность через точку 2.5 

### nltk NER

In [303]:
def nltk_ner(text):
    ne_tree = ne_chunk(pos_tag(word_tokenize(text)), binary=True)
    named_entities = [ " ".join(w for w, t in elt) for elt in ne_tree if isinstance(elt, nltk.Tree)]
    return set(named_entities)

In [310]:
%%time
fin_df['nltk_ner'] = fin_df['text_without_tags'].apply(nltk_ner)

CPU times: user 23min 33s, sys: 1.81 s, total: 23min 34s
Wall time: 23min 34s


In [30]:
testext = """GAM is awesome for this. We\'ve also had good luck with Google Apps Scripts. The advantage is that it makes it marginally easier to set up credentials: it does so behind the scenes so you never have creds with admin permissions hit disk. The disadvantage is you\'re now editing an old-ish version of JavaScript in a janky browser editor that saves to some weird bespoke JSON format. You can also use clasp (<a href="https://github.com/google/clasp" rel="nofollow">https://github.com/google/clasp</a>) to edit these files from your favorite editor but now you own a Software Engineering Project.<p>Here\'s roughly what that looks like to get all the mentioned info out:<p><pre><code>    function getGroups(domain) {\n       var groups = [], pageToken, page;\n       do {\n           page = AdminDirectory.Groups.list({\n               domain: domain,\n               maxResults: 100,\n               pageToken: pageToken\n           });\n    \n           if (page.groups && page.groups.length > 0) {\n               groups = groups.concat(page.groups);\n           }\n           pageToken = page.nextPageToken;\n    \n       } while (pageToken);\n       return groups;\n    }\n    \n    function getGroupPerms(groups) {\n     var perms = [];\n     for (var i = 0; i < groups.length; i++) {\n       perms.push(AdminGroupsSettings.Groups.get(groups[i].email));\n     }\n     return perms;\n    }\n</code></pre>\nDisclaimer: I\'m the author of the linked blog post."""


from nltk.tag import StanfordNERTagger

st = StanfordNERTagger('/usr/share/stanford-ner/classifiers/english.all.3class.distsim.crf.ser.gz',
                       '/usr/share/stanford-ner/stanford-ner.jar',
                        encoding='utf-8')

named_entities = []

classified_text = st.tag(word_tokenize(testext))

for word, cat in classified_text:
    if cat != 'O':
        named_entities.append(word)

## промежуточные результаты

In [337]:
# все три способа иногда находят чуть-чуть разное

In [347]:
fin_df.loc[7,['text_without_tags', 'spacy_ents', 'prop_names_spacy', 'nltk_ner']].reset_index().T

,0,1,2,3
index,text_without_tags,spacy_ents,prop_names_spacy,nltk_ner
7,"(Retyped is part of the Bridge.NET C# -> JS/TypeScript converter) It’s super exciting too see Bridge maintaining its place in a world that includes both C# -> WebAsm and mature TypeScript ecosystems. I’d started to wonder if Bridge still had a role to play now that those two ecosystems are maturing, but I think the marshalling cost between WebAsm and JS/Dom is likely to discourage full WebAsm implementations in many cases, and if it does the ability to do seamless code sharing between frontend TypeScript and backend C# is pretty killer.","{NET, TypeScript, WebAsm, two, Dom, Retyped, JS/, JS/TypeScript, Bridge}","{NET, TypeScript, WebAsm, Dom, JS, C, Bridge}","{WebAsm, TypeScript, Retyped, Bridge}"


### здесь должна быть ручная разметка и дообучение spacy

______________________

## Исследование spacy NER из коробки на данных: 

In [ ]:
# все тексты загнала в один док

In [67]:
the_biggest_text_ever = ' '.join(fin_df['full_text'].astype(str).values)

In [68]:
len(the_biggest_text_ever)

77021890

In [ ]:
# запихала его в spacy

In [72]:
doc = nlp(the_biggest_text_ever[:1000000-1])

In [100]:
res= defaultdict(lambda: defaultdict(int))

In [ ]:
# считаю сколько раз встретится какая-либо сущность с каким-либо лейблом

In [101]:
for ent in doc.ents:
    res[ent.text][ent.label_]+=1

In [123]:
another_dict = defaultdict(lambda: set())
for ent in doc.ents:
    another_dict[ent.label_] = another_dict[ent.label_] | set([ent.text])

### найденные лейблы сущностей
#### топ 3: Person, Org, GPE

In [126]:
ents_cnt = defaultdict(int)
for k,v in res.items():
    for k1, v1 in v.items():
        ents_cnt[k1] += v1

In [127]:
ents_cnt

defaultdict(int,
            {'CARDINAL': 924,
             'DATE': 738,
             'EVENT': 20,
             'FAC': 66,
             'GPE': 1369,
             'LANGUAGE': 18,
             'LAW': 32,
             'LOC': 142,
             'MONEY': 201,
             'NORP': 298,
             'ORDINAL': 200,
             'ORG': 2826,
             'PERCENT': 97,
             'PERSON': 1333,
             'PRODUCT': 162,
             'QUANTITY': 68,
             'TIME': 98,
             'WORK_OF_ART': 106})

## все найденные сущности в порядке убывания их встречаемости И все их лейблы

In [ ]:
# например, в топе есть Java JavaScript MySQL Microsoft

In [141]:
# python как-то нехорошо нашелся
res['Python']

defaultdict(int, {'ORG': 1})

In [138]:
for big_word in sorted(res, key=lambda key: sum(res[key].values()))[::-1]:
    if not big_word.isdigit() and 'http' not in big_word and '<' not in big_word and list(res[big_word].keys())!=['GPE']:
        print(big_word, '\t\t', list(res[big_word].keys()))
        

one 		 ['CARDINAL']
Android 		 ['PRODUCT', 'PERSON', 'GPE', 'ORG']
EU 		 ['GPE', 'ORG']
iPhone 		 ['PERSON', 'ORG']
first 		 ['ORDINAL']
Google 		 ['PRODUCT', 'ORG']
Apple 		 ['ORG']
GDPR 		 ['NORP', 'PERSON', 'LAW', 'WORK_OF_ART', 'FAC', 'GPE', 'ORG']
two 		 ['QUANTITY', 'CARDINAL']
COBOL 		 ['PERSON', 'GPE', 'ORG']
HN 		 ['GPE', 'ORG']
  		 ['ORDINAL', 'PERSON', 'NORP', 'ORG']
Facebook 		 ['PERSON', 'GPE', 'ORG']
One 		 ['CARDINAL']
WhatsApp 		 ['ORG']
AI 		 ['PRODUCT', 'PERSON', 'WORK_OF_ART', 'DATE', 'GPE']
HTTP 		 ['LOC', 'NORP', 'ORG']
European 		 ['NORP']
years 		 ['DATE']
POST 		 ['ORG']
today 		 ['DATE']
second 		 ['ORDINAL']
Uber 		 ['PERSON', 'GPE', 'ORG']
IBM 		 ['ORG']
OS 		 ['ORG']
GET 		 ['ORG']
Java 		 ['PERSON', 'NORP', 'ORG']
Europe 		 ['LOC']
YouTube 		 ['PERSON', 'NORP', 'ORG']
third 		 ['ORDINAL']
Watson 		 ['PERSON', 'GPE']
GC 		 ['ORG']
100% 		 ['PERCENT']
iPhones 		 ['PRODUCT', 'GPE', 'ORG']

   		 ['ORDINAL', 'PERSON', 'ORG']
UML 		 ['ORG']
half 		 ['CARDINAL']

every single penny 		 ['MONEY']
10}0.5 		 ['CARDINAL']
Paaart – free 		 ['PERSON']
60b 		 ['MONEY']
every hour 		 ['TIME']
My iDevices 		 ['ORG']
e2e 		 ['NORP']
120FPS 		 ['CARDINAL']
Safari (& 		 ['ORG']
Drupalgeddon3 		 ['PERSON']
Apple Store 		 ['ORG']
Envato 		 ['ORG']
27k 		 ['MONEY']
less than a year ago 		 ['DATE']
Law Procedurals 		 ['WORK_OF_ART']
Show HN: Profile visualization for GitHub 		 ['WORK_OF_ART']
Eastern European 		 ['NORP']
Blazor 0.2.0 		 ['PRODUCT']
Prolog 		 ['ORG']
SAAS 		 ['ORG']
FB and Messenger 		 ['WORK_OF_ART']
Bumblefuck 		 ['PERSON']
hundreds of millions 		 ['CARDINAL']
Wayback 		 ['ORG']
10 cent 		 ['MONEY']
Paul Fuqua 		 ['PERSON']
Pullman 		 ['PERSON']
55-63 		 ['DATE']
a 24 hour 		 ['TIME']
age 65 		 ['DATE']
Inno. 		 ['ORG']
Architects 		 ['ORG']
larger than a dozen 		 ['CARDINAL']
Angie 		 ['PERSON']
a few thousand dollars 		 ['MONEY']
500ms 		 ['ORG']
new Constitutional Amendment 		 ['LAW']
Slacking 		 ['PERSON']
fourth 		 ['ORDINAL']
Ancient For

ARP 		 ['ORG']
WeMos 		 ['ORG']
I-Flex 		 ['PRODUCT']
Disney 		 ['ORG']

           		 ['PERSON']
Defense 		 ['ORG']
about 3 or 4 years 		 ['DATE']
4.5 		 ['CARDINAL']
p>+ Investing 		 ['ORG']
iPhone 7 		 ['PRODUCT']
28 years 		 ['DATE']
Boeings 		 ['ORG']
Times for Alternatives to Einstein’s Theory of Gravity 		 ['ORG']
GF 		 ['ORG']
the National Science Foundation 		 ['ORG']
S-Corps 		 ['ORG']
Element B 		 ['ORG']
Aspinwall Mountain Wear 		 ['ORG']
another 6 weeks 		 ['DATE']
Wednesday 		 ['DATE']
AutoHotkey 		 ['ORG']
3-month 		 ['ORDINAL']
ESP8266 		 ['PERSON']
“Ballmer Laughs at iPhone” 		 ['WORK_OF_ART']
> > 		 ['TIME']
MY 		 ['ORG']
next day 		 ['DATE']
The Elite of Poland Blown 		 ['ORG']
Intuit 		 ['NORP']
the last hundred years 		 ['DATE']
1yr 		 ['ORDINAL']
UBI 		 ['ORG']
Paul Graham 		 ['PERSON']
Handbook Think 		 ['PERSON']
Attacker Dilemma 		 ['PERSON']
the Fugitive Slave Act 		 ['LAW']
twentieth century 		 ['DATE']
Jack Lew 		 ['PERSON']
next month 		 ['DATE']
a few week

### забавный пример
* PERSON Junior Developer
* WORK_OF_ART The Senior Developers

In [62]:
txt[3]\


' The typical explanation (a la r/programming) seems to be something or other about webdevs being naturally impatient, faddish and incompetent</i> Okay, let\'s look at the facts, I\'m open to any explanation but this is the most convincing one.  2.5 years for the youngest isn’t that old in the scheme of things</i> 2.5 years is NOTHING. N O T H I N G. CPlusPlus98 clocks in at 20 years now and it\'s one of the most stable drivers of software. HTML is 25 years old and basic websites from back in the day still work. You are churning through frameworks an order of magnitude too fast!  2.5 years for the youngest isn’t that old in the scheme of things [...] but it’s still a ways off our caricature.</i> This sentence alone is a caricature and the fact you cannot see that makes the caricature even worse.  The advantage of this architecture is that you can easily adapt as new practices emerge, which makes sense at a time of rapid innovation (like the past few years). The disadvantage is that it 

In [63]:
for ent in nlp(txt[3]).ents:
    print(ent.label_, ent.text)

CARDINAL one
DATE 2.5 years
DATE 2.5 years
DATE 20 years
DATE 25 years old
DATE the day
DATE 2.5 years
DATE the past few years
ORG microlibs.</i
DATE lifetime
ORG  
ORG JavaScript
CARDINAL 1
ORG NPM
CARDINAL one
PRODUCT JavaScript
PRODUCT JavaScript
PERSON Frankenstein
PERSON Github
ORG JavaScript
ORG Software
TIME the hours
ORG Google
GPE Medium
PERSON X.js
DATE the day
CARDINAL one
EVENT Twitter
PERSON Y.js
DATE first year ComSci
LOC Smalltalk
PRODUCT JavaScript
ORG OOP
ORG JavaScript
MONEY $50 dollar
PRODUCT JavaScript
CARDINAL Nineteen thousand
CARDINAL Nineteen thousand
CARDINAL one
PERSON Junior Developer
WORK_OF_ART The Senior Developers
ORG JavaScript
PRODUCT JavaScript
ORG Front-end
